In [1]:
import pandas as pd

## Process 2022

In [ ]:
fare_22 = pd.read_excel("../../raw_data/2022 Fare Revenue.xlsx", sheet_name="2022 Fare Revenue")
fare_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6086 entries, 0 to 6085
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   State/Parent NTD ID      3054 non-null   object 
 1   NTD ID                   6086 non-null   int64  
 2   Agency Name              6086 non-null   object 
 3   Reporter Type            6032 non-null   object 
 4   Reporting Module         6086 non-null   object 
 5   Mode                     6086 non-null   object 
 6   TOS                      6086 non-null   object 
 7   Expense Type             6086 non-null   object 
 8   Passenger Paid Fares     4135 non-null   float64
 9   Organization Paid Fares  2511 non-null   float64
 10  Total Fares              6086 non-null   int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 523.1+ KB


In [3]:
# Remove unneeded columns.
fare_22.drop(columns=['State/Parent NTD ID', 'TOS', 'Reporting Module'], inplace=True)
fare_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6086 entries, 0 to 6085
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NTD ID                   6086 non-null   int64  
 1   Agency Name              6086 non-null   object 
 2   Reporter Type            6032 non-null   object 
 3   Mode                     6086 non-null   object 
 4   Expense Type             6086 non-null   object 
 5   Passenger Paid Fares     4135 non-null   float64
 6   Organization Paid Fares  2511 non-null   float64
 7   Total Fares              6086 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 380.5+ KB


In [4]:
# Remove the "reporter" suffix from the entries in reporter type.
fare_22['Reporter Type'] = fare_22['Reporter Type'].fillna('Null')
fare_22['Reporter Type'] = [words[0] for words in (item.split() for item in fare_22['Reporter Type'])]
fare_22['Reporter Type'] = fare_22['Reporter Type'].replace('Null', None)
fare_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6086 entries, 0 to 6085
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NTD ID                   6086 non-null   int64  
 1   Agency Name              6086 non-null   object 
 2   Reporter Type            6032 non-null   object 
 3   Mode                     6086 non-null   object 
 4   Expense Type             6086 non-null   object 
 5   Passenger Paid Fares     4135 non-null   float64
 6   Organization Paid Fares  2511 non-null   float64
 7   Total Fares              6086 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 380.5+ KB


In [5]:
# Isolate only funds-earned during period columns.
fare_22_earned = fare_22[fare_22['Expense Type'] == "Funds Earned During Period"]
fare_22_earned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1348 entries, 0 to 6057
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NTD ID                   1348 non-null   int64  
 1   Agency Name              1348 non-null   object 
 2   Reporter Type            1308 non-null   object 
 3   Mode                     1348 non-null   object 
 4   Expense Type             1348 non-null   object 
 5   Passenger Paid Fares     1202 non-null   float64
 6   Organization Paid Fares  542 non-null    float64
 7   Total Fares              1348 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 94.8+ KB


In [6]:
# Aggregate by agency and mode type.
grp_22 = fare_22_earned.drop(columns=['Expense Type', "Agency Name", "Reporter Type"]).groupby(by=['NTD ID', 'Mode'])
fare_22_clean = grp_22.sum().reset_index()
fare_22_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NTD ID                   1173 non-null   int64  
 1   Mode                     1173 non-null   object 
 2   Passenger Paid Fares     1173 non-null   float64
 3   Organization Paid Fares  1173 non-null   float64
 4   Total Fares              1173 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 45.9+ KB


In [7]:
# Re-add Agency Name and Reporter Type
name_map_dict = pd.Series(data=fare_22_earned['Agency Name'].values, index=fare_22_earned['NTD ID']).to_dict()
def id_to_name(id):
    return name_map_dict[id]

rep_map_dict = pd.Series(data=fare_22_earned['Reporter Type'].values, index=fare_22_earned['NTD ID']).to_dict()
def id_to_rep_type(id):
    return rep_map_dict[id]

fare_22_clean.insert(1, column='Agency Name', value= fare_22_clean['NTD ID'].apply(id_to_name))
fare_22_clean.insert(2, column='Reporter Type', value= fare_22_clean['NTD ID'].apply(id_to_rep_type))
fare_22_clean.insert(2, column='Year', value=[2022]*len(fare_22_clean['NTD ID']))
fare_22_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NTD ID                   1173 non-null   int64  
 1   Agency Name              1173 non-null   object 
 2   Year                     1173 non-null   int64  
 3   Reporter Type            1140 non-null   object 
 4   Mode                     1173 non-null   object 
 5   Passenger Paid Fares     1173 non-null   float64
 6   Organization Paid Fares  1173 non-null   float64
 7   Total Fares              1173 non-null   int64  
dtypes: float64(2), int64(3), object(3)
memory usage: 73.4+ KB


## Clean 2021 - 2019

In [8]:
def clean_21_to_19(df:pd.DataFrame, year:int):
    df['NTD ID'] = [str(num)[-5:] for num in df['NTD ID']]
    df = df.drop(columns=['Reporting Module', 'TOS'])
    # Remove the "reporter" suffix from the entries in reporter type.
    df['Reporter Type'] = df['Reporter Type'].fillna('Null')
    df['Reporter Type'] = [words[0] for words in (item.split() for item in df['Reporter Type'])]
    df['Reporter Type'] = df['Reporter Type'].replace('Null', None)
    # Isolate earnings rows.
    df = df[df['Expense Type'] == "Funds Earned During Period"]
    # Group and aggregate by agency and mode type.
    df_grp = df.drop(columns=['Expense Type', "Agency Name", "Reporter Type"]).groupby(by=['NTD ID', 'Mode'])
    df_clean = df_grp.sum().reset_index()
    #Re-add Agency Name and Reporter Type
    name_map_dict = pd.Series(data=df['Agency Name'].values, index=df['NTD ID']).to_dict()
    def id_to_name(id):
        return name_map_dict[id]

    rep_map_dict = pd.Series(data=df['Reporter Type'].values, index=df['NTD ID']).to_dict()
    def id_to_rep_type(id):
        return rep_map_dict[id]
    
    df_clean.insert(1, column='Agency Name', value= df_clean['NTD ID'].apply(id_to_name))
    df_clean.insert(2, column='Reporter Type', value= df_clean['NTD ID'].apply(id_to_rep_type))
    # Add a year value.
    df_clean.insert(2, column='Year', value=[year]*len(df_clean['NTD ID']))
    # Make NTD ID an integer value.
    df_clean['NTD ID'] = df_clean['NTD ID'].astype(dtype='int64')
    # Return cleaned data.
    return df_clean

In [ ]:
fare_21 = pd.read_excel("../../raw_data/2021 Fare Revenue.xlsx")
fare_20 = pd.read_excel('../../raw_data/2020 Fare Revenue.xlsx')
fare_19 = pd.read_excel('../../raw_data/2019 Fare Revenue.xlsx')
frames = [fare_21, fare_20, fare_19]
years = [2021, 2020, 2019]
cleaned = []
for df, yr in zip(frames, years):
    cleaned.append(clean_21_to_19(df, yr))

dat_21_19 = pd.concat(cleaned)
dat_21_19.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3559 entries, 0 to 1217
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   NTD ID                   3559 non-null   int64 
 1   Agency Name              3559 non-null   object
 2   Year                     3559 non-null   int64 
 3   Reporter Type            3559 non-null   object
 4   Mode                     3559 non-null   object
 5   Passenger Paid Fares     3559 non-null   int64 
 6   Organization Paid Fares  3559 non-null   int64 
 7   Total Fares              3559 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 250.2+ KB


## Clean 2018
It is almost identical to 2021 - 2019 except that it has a Legacy NTD ID column.

In [ ]:
fare_18 = pd.read_excel('../../raw_data/2018 Fare Revenue.xlsx')
fare_18.drop(columns=['Legacy NTD ID'], inplace=True)
fare_18_clean = clean_21_to_19(fare_18, 2018)
fare_18_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   NTD ID                   1234 non-null   int64 
 1   Agency Name              1234 non-null   object
 2   Year                     1234 non-null   int64 
 3   Reporter Type            1234 non-null   object
 4   Mode                     1234 non-null   object
 5   Passenger Paid Fares     1234 non-null   int64 
 6   Organization Paid Fares  1234 non-null   int64 
 7   Total Fares              1234 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 77.3+ KB


## Clean 2017 and 2016

In [ ]:
fare_17 = pd.read_excel('../../raw_data/2017 Fare Revenue.xlsx')
fare_17['Total Fares'] = fare_17['Fares']
fare_17.drop(columns=['Fares', 'Legacy NTD ID'], inplace=True)
fare_17_clean = clean_21_to_19(fare_17, 2017)
fare_17_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NTD ID         1253 non-null   int64  
 1   Agency Name    1253 non-null   object 
 2   Year           1253 non-null   int64  
 3   Reporter Type  1253 non-null   object 
 4   Mode           1253 non-null   object 
 5   Total Fares    1253 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 58.9+ KB


In [ ]:
fare_16 = pd.read_excel('../../raw_data/2016 Fare Revenue.xlsx')
fare_16['Total Fares'] = fare_16['Fares']
fare_16.insert(0, 'NTD ID', value=fare_16['5 Digit NTD ID'])
fare_16.drop(columns=['5 Digit NTD ID', 'Fares', 'Legacy NTD ID'], inplace=True)
fare_16_clean = clean_21_to_19(fare_16, 2016)
fare_16_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1249 entries, 0 to 1248
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NTD ID         1249 non-null   int64  
 1   Agency Name    1249 non-null   object 
 2   Year           1249 non-null   int64  
 3   Reporter Type  1249 non-null   object 
 4   Mode           1249 non-null   object 
 5   Total Fares    1249 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 58.7+ KB


## Combine All Datasets

In [13]:
frames = [fare_22_clean, dat_21_19, fare_18_clean, fare_17_clean, fare_16_clean]
all_dat = pd.concat(frames)
all_dat['NTD ID'] = [f"{num:05d}" for num in all_dat['NTD ID']]
all_dat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8468 entries, 0 to 1248
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   NTD ID                   8468 non-null   object 
 1   Agency Name              8468 non-null   object 
 2   Year                     8468 non-null   int64  
 3   Reporter Type            8435 non-null   object 
 4   Mode                     8468 non-null   object 
 5   Passenger Paid Fares     5966 non-null   float64
 6   Organization Paid Fares  5966 non-null   float64
 7   Total Fares              8468 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 595.4+ KB


In [ ]:
all_dat.to_csv('../../data/NTD_Fare_Revenue.csv', index=False, sep=',')